In [10]:
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from matplotlib.offsetbox import (TextArea, AnnotationBbox)
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.datasets import fetch_california_housing
from scipy.stats import randint, uniform


In [11]:
install_mapie = False
if install_mapie is True:
    !pip install git+https://github.com/scikit-learn-contrib/MAPIE.git@add_prefit_cqr

In [3]:
from mapie.metrics import (
    regression_coverage_score,
    regression_mean_width_score
    )
from mapie.quantile_regression import MapieQuantileRegressor

In [4]:
data = fetch_california_housing(as_frame=True)
X = pd.DataFrame(data=data.data, columns=data.feature_names)
y = pd.DataFrame(data=data.target)*100

In [5]:
y = y['MedHouseVal']
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
)
X_train, X_calib, y_train, y_calib = train_test_split(
    X_train,
    y_train,
)

In [6]:
list_estimators = []
estimator_low = LGBMRegressor(
    objective='quantile',
    alpha=0.05,
)
estimator_low.fit(X_train, y_train)
list_estimators.append(estimator_low)

estimator_high = LGBMRegressor(
    objective='quantile',
    alpha=0.95,
)
estimator_high.fit(X_train, y_train)
list_estimators.append(estimator_high)


estimator = LGBMRegressor(
    objective='quantile',
    alpha=0.5,
)
estimator.fit(X_train, y_train)
list_estimators.append(estimator)


In [7]:
mapie = MapieQuantileRegressor(list_estimators, cv="prefit")
mapie.fit(X_calib, y_calib)
prefit_predict = mapie.predict(X_test)

/Users/llacombe/CODE/MAPIE/mapie/quantile_regression.py:462: UserWarning: WARNING: The alpha that is set needs to be the same as the alpha of your prefitted model in the following order [alpha/2, 1 - alpha/2, 0.5]
  warnings.warn(
/Users/llacombe/CODE/MAPIE/mapie/utils.py:466: UserWarning: WARNING: The initial prediction values from the quantile method
present issues as the upper quantile values might be higher than the
lower quantile values.
  warnings.warn(
/Users/llacombe/CODE/MAPIE/mapie/utils.py:479: UserWarning: WARNING: Following the additional value added to have conformal predictions, the upper and lower bound present issues as one might be higher or lower than the other.
  warnings.warn(


In [8]:
mapie = MapieQuantileRegressor(estimator)
mapie.fit(X_train, y_train, X_calib=X_calib, y_calib=y_calib)
split_predict = mapie.predict(X_test)

In [9]:
for i in range(len(prefit_predict)):
    assert (prefit_predict[i]==split_predict[i]).all()